[Esteban Munoz](emunozh@gmail.com)

Fri Jul 24

In [1]:
import pandas as pd

In [2]:
xls = pd.ExcelFile('raw/Zensus-Standardauswertung Hamburger Statistische Gebiete nach SAFE_Seller.xlsx')

In [3]:
def formatIndex(Int):
    Str = str(int(Int))
    n = len(Str)
    if n < 6:
        return "0"*(6-n) + Str
    else:
        return Str

In [4]:
def deleteExtraCharacters(Val):
    Val = str(Val)
    Val = Val.replace("(","")
    Val = Val.replace(")","")
    return float(Val)

In [5]:
def makeNew(df, idx='InsgesaRAUM_newmt', col='Stat. Geb.', cut=11, col_id=9):
    columns_groups = df.index[0:cut].tolist()
    #columns_groups.insert(0, 'Insgesamt')
    df_new = pd.DataFrame(
        index = [formatIndex(Val) for Val in df.loc[idx,col].tolist()],
        columns = columns_groups)
    for cgroup in columns_groups:
        df_new.loc[:,cgroup] = [deleteExtraCharacters(a) for a in df.loc[cgroup, col_id].tolist()]
    df_new.ix[0,0] = col_id
    return df_new

# (1) BArt

In [6]:
Bart = xls.parse(
    0, na_values=['-','.'],
    skiprows=[0,1,2,3,4,5,7,8],
    #converters={4:deleteExtraCharacters},
    skip_footer=6,
    parse_cols=[0,2,3,4,5,6,7])

In [7]:
# for some reason pandas is not getting this data type right
Bart.Wohnheime = Bart.Wohnheime.astype(float)

In [8]:
# format index
Bart["Unnamed: 0"] = Bart["Unnamed: 0"].astype(float)
Bart["Unnamed: 0"] = [formatIndex(Val) for Val in Bart["Unnamed: 0"].tolist()]
new_columns = Bart.columns.values; new_columns[0] = 'Stat. Geb.'; Bart.columns = new_columns
Bart = Bart.set_index("Stat. Geb.")

In [9]:
Bart.dtypes

Insgesamt                        float64
Wohngebäude                      float64
Wohngebäude (ohne Wohnheime)     float64
Wohnheime                        float64
Sonstige Gebäude mit Wohnraum    float64
dtype: object

In [10]:
Bart.head() 

,Insgesamt,Wohngebäude,Wohngebäude (ohne Wohnheime),Wohnheime,Sonstige Gebäude mit Wohnraum
Stat. Geb.,,,,,
001002,9,NaN,NaN,NaN,8
001003,6,NaN,NaN,NaN,5
001004,7,3,3,NaN,5
001005,6,3,3,NaN,4
001006,15,3,3,NaN,13


In [11]:
Bart.tail()

,Insgesamt,Wohngebäude,Wohngebäude (ohne Wohnheime),Wohnheime,Sonstige Gebäude mit Wohnraum
Stat. Geb.,,,,,
107011,300,299,299,NaN,NaN
107012,148,146,146,NaN,3
107013,215,215,215,NaN,NaN
107014,413,412,412,NaN,NaN
107015,517,517,517,NaN,NaN


In [12]:
Bart.to_csv("BART.csv")

# (2) BJA

In [13]:
BJA = xls.parse(
    1, index_col=[1], na_values=['-','.'],
    skiprows=[0,1,2,3,4,5],
    skip_footer=26
)

In [14]:
BJA_new = makeNew(BJA, cut=11)

In [15]:
BJA_new.dtypes

Insgesamt          float64
Vor 1919           float64
1919 - 1948        float64
1949 - 1978        float64
1979 - 1986        float64
1987 - 1990        float64
1991 - 1995        float64
1996 - 2000        float64
2001 - 2004        float64
2005 - 2008        float64
2009 und später    float64
dtype: object

In [16]:
BJA_new.head()

,Insgesamt,Vor 1919,1919 - 1948,1949 - 1978,1979 - 1986,1987 - 1990,1991 - 1995,1996 - 2000,2001 - 2004,2005 - 2008,2009 und später
001002,9,4,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
001003,6,NaN,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
001004,7,4,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
001005,6,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
001006,15,8,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
BJA_new.tail()

,Insgesamt,Vor 1919,1919 - 1948,1949 - 1978,1979 - 1986,1987 - 1990,1991 - 1995,1996 - 2000,2001 - 2004,2005 - 2008,2009 und später
107011,300,NaN,NaN,8,NaN,NaN,31,NaN,7,NaN,NaN
107012,148,NaN,NaN,NaN,NaN,21,NaN,NaN,NaN,NaN,NaN
107013,215,NaN,6,37,36,18,NaN,NaN,NaN,NaN,NaN
107014,413,NaN,NaN,NaN,162,36,NaN,30,22,21,NaN
107015,517,3,4,17,14,36,NaN,NaN,26,NaN,NaN


In [18]:
BJA_new.to_csv("BJA.csv")

#(3) Ownership

In [19]:
OWN = xls.parse(
    2, index_col=[1], na_values=['-','.'],
    skiprows=[0,1,2,3,4,5],
    skip_footer=22
)

In [20]:
OWN_new = makeNew(OWN, cut=9)

In [21]:
OWN_new.dtypes

Insgesamt                                       float64
Gemeinschaft von Wohnungseigentümern/-innen     float64
Privatperson/-en                                float64
Wohnungsgenossenschaft                          float64
Kommune oder kommunales Wohnungsunternehmen     float64
Privatwirtschaftliches Wohnungsunternehmen      float64
Anderes privatwirtschaftliches Unternehmen      float64
Bund oder Land                                  float64
Organisation ohne Erwerbszweck (z.B. Kirche)    float64
dtype: object

In [22]:
OWN_new.head()

,Insgesamt,Gemeinschaft von Wohnungseigentümern/-innen,Privatperson/-en,Wohnungsgenossenschaft,Kommune oder kommunales Wohnungsunternehmen,Privatwirtschaftliches Wohnungsunternehmen,Anderes privatwirtschaftliches Unternehmen,Bund oder Land,Organisation ohne Erwerbszweck (z.B. Kirche)
001002,9,NaN,5,NaN,NaN,NaN,NaN,NaN,3
001003,6,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN
001004,7,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN
001005,6,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN
001006,15,5,6,NaN,NaN,NaN,3,NaN,NaN


In [23]:
OWN_new.tail()

,Insgesamt,Gemeinschaft von Wohnungseigentümern/-innen,Privatperson/-en,Wohnungsgenossenschaft,Kommune oder kommunales Wohnungsunternehmen,Privatwirtschaftliches Wohnungsunternehmen,Anderes privatwirtschaftliches Unternehmen,Bund oder Land,Organisation ohne Erwerbszweck (z.B. Kirche)
107011,300,3,294,NaN,NaN,NaN,NaN,NaN,NaN
107012,148,7,59,47,15,19,NaN,NaN,NaN
107013,215,13,19,NaN,15,60,NaN,NaN,NaN
107014,413,3,309,NaN,NaN,NaN,NaN,NaN,4
107015,517,15,444,6,5,47,NaN,NaN,NaN


In [24]:
OWN_new.to_csv("OWN.csv")

#(4) Btyp

In [25]:
Btyp = xls.parse(
    3, index_col=[1], na_values=['-','.'],
    skiprows=[0,1,2,3,4,5],
    skip_footer=14
)

In [26]:
Btyp_new = makeNew(Btyp, cut=5)

In [27]:
Btyp_new.dtypes

Insgesamt             float64
Freistehendes Haus    float64
Doppelhaushälfte      float64
Gereihtes Haus        float64
Anderer Gebäudetyp    float64
dtype: object

In [28]:
Btyp_new.head()

,Insgesamt,Freistehendes Haus,Doppelhaushälfte,Gereihtes Haus,Anderer Gebäudetyp
001002,9,NaN,NaN,NaN,6
001003,6,NaN,NaN,3,3
001004,7,NaN,NaN,3,3
001005,6,NaN,NaN,5,NaN
001006,15,3,NaN,6,7


In [29]:
Btyp_new.tail()

,Insgesamt,Freistehendes Haus,Doppelhaushälfte,Gereihtes Haus,Anderer Gebäudetyp
107011,300,72,8,217,3
107012,148,14,NaN,112,17
107013,215,8,6,122,79
107014,413,12,16,380,5
107015,517,228,29,226,34


In [30]:
Btyp_new.to_csv("BTYP.csv")

#(5) NDU

In [31]:
Ndu = xls.parse(
    4, index_col=[1], na_values=['-','.'],
    skiprows=[0,1,2,3,4,5],
    skip_footer=16
)

In [32]:
Ndu_new = makeNew(Ndu, cut=6)

In [33]:
Ndu_new.dtypes

Insgesamt                float64
1 Wohnung                float64
2 Wohnungen              float64
3 - 6 Wohnungen          float64
7 - 12 Wohnungen         float64
13 und mehr Wohnungen    float64
dtype: object

In [34]:
Ndu_new.head()

,Insgesamt,1 Wohnung,2 Wohnungen,3 - 6 Wohnungen,7 - 12 Wohnungen,13 und mehr Wohnungen
001002,9,8,NaN,NaN,NaN,NaN
001003,6,4,3,NaN,NaN,NaN
001004,7,3,NaN,NaN,NaN,NaN
001005,6,3,NaN,NaN,3,NaN
001006,15,8,3,4,NaN,NaN


In [35]:
Ndu_new.tail()

,Insgesamt,1 Wohnung,2 Wohnungen,3 - 6 Wohnungen,7 - 12 Wohnungen,13 und mehr Wohnungen
107011,300,287,12,NaN,NaN,NaN
107012,148,45,NaN,40,61,NaN
107013,215,60,3,70,79,4
107014,413,398,6,7,3,NaN
107015,517,451,64,3,NaN,NaN


In [36]:
Ndu_new.to_csv("NDU.csv")

#(6) HEAT

In [37]:
HEAT = xls.parse(
    5, index_col=[1], na_values=['-','.'],
    skiprows=[0,1,2,3,4,5],
    skip_footer=18
)

In [38]:
HEAT_new = makeNew(HEAT, cut=7)

In [39]:
HEAT_new.dtypes

Insgesamt                                           float64
Fernheizung (Fernwärme)                             float64
Etagenheizung                                       float64
Blockheizung                                        float64
Zentralheizung                                      float64
Einzel-/Mehrraumöfen (auch Nachtspeicherheizung)    float64
Keine Heizung im Gebäude oder in den Wohnungen      float64
dtype: object

In [40]:
HEAT_new.head()

,Insgesamt,Fernheizung (Fernwärme),Etagenheizung,Blockheizung,Zentralheizung,Einzel-/Mehrraumöfen (auch Nachtspeicherheizung),Keine Heizung im Gebäude oder in den Wohnungen
001002,9,3,NaN,NaN,5,NaN,NaN
001003,6,3,NaN,NaN,NaN,NaN,NaN
001004,7,3,NaN,NaN,4,NaN,NaN
001005,6,4,3,NaN,NaN,NaN,NaN
001006,15,8,NaN,NaN,6,NaN,NaN


In [41]:
HEAT_new.tail()

,Insgesamt,Fernheizung (Fernwärme),Etagenheizung,Blockheizung,Zentralheizung,Einzel-/Mehrraumöfen (auch Nachtspeicherheizung),Keine Heizung im Gebäude oder in den Wohnungen
107011,300,25,34,6,231,4,NaN
107012,148,24,14,16,90,4,NaN
107013,215,59,3,27,124,NaN,NaN
107014,413,108,39,13,242,11,NaN
107015,517,38,51,10,408,10,NaN


In [42]:
HEAT_new.to_csv("HEAT.csv")

#(7) NUTZ

In [43]:
NUTZ = xls.parse(
    6, index_col=[1], na_values=['-','.'],
    skiprows=[0,1,2,3,4,5,6],
    skip_footer=16
)

In [44]:
NUTZ_new = makeNew(NUTZ, cut=6, col_id=10, col="Stat.Geb")

In [45]:
NUTZ_new.head()

,Insgesamt,Von Eigentümer/-in bewohnt,Zu Wohnzwecken vermietet (auch mietfrei),Ferien- und Freizeitwohnung,Leer stehend,Diplomaten-/Streitkräftewohnung
001002,10,NaN,6,NaN,3,NaN
001003,7,3,3,NaN,NaN,NaN
001004,32,3,30,NaN,NaN,NaN
001005,51,NaN,39,NaN,11,NaN
001006,41,4,30,NaN,7,NaN


In [46]:
NUTZ_new.tail()

,Insgesamt,Von Eigentümer/-in bewohnt,Zu Wohnzwecken vermietet (auch mietfrei),Ferien- und Freizeitwohnung,Leer stehend,Diplomaten-/Streitkräftewohnung
107011,314,289,23,NaN,3,NaN
107012,812,64,747,NaN,NaN,NaN
107013,1153,NaN,1142,NaN,3,NaN
107014,464,275,189,NaN,NaN,NaN
107015,586,450,136,NaN,NaN,NaN


In [47]:
NUTZ_new.to_csv("NUTZ.csv")

#(8) SQM

In [48]:
SQM = xls.parse(
    7, index_col=[1], na_values=['-','.'],
    skiprows=[0,1,2,3,4,5,6],
    skip_footer=26
)

In [49]:
SQM_new = makeNew(SQM, cut=11, col_id=10, col="Stat. Geb")

In [50]:
SQM_new.dtypes

Insgesamt       float64
Unter 40        float64
40 - 59         float64
60 - 79         float64
80 - 99         float64
100 - 119       float64
120 - 139       float64
140 - 159       float64
160 - 179       float64
180 - 199       float64
200 und mehr    float64
dtype: object

In [51]:
SQM_new.head()

,Insgesamt,Unter 40,40 - 59,60 - 79,80 - 99,100 - 119,120 - 139,140 - 159,160 - 179,180 - 199,200 und mehr
001002,10,3,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
001003,7,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
001004,32,NaN,4,14,6,5,3,NaN,NaN,NaN,NaN
001005,51,6,14,24,3,4,NaN,NaN,NaN,NaN,NaN
001006,41,3,10,8,10,4,NaN,3,3,3,NaN


In [52]:
SQM_new.tail()

,Insgesamt,Unter 40,40 - 59,60 - 79,80 - 99,100 - 119,120 - 139,140 - 159,160 - 179,180 - 199,200 und mehr
107011,314,3,7,10,46,88,122,22,6,5,6
107012,812,4,241,309,201,35,13,NaN,NaN,NaN,NaN
107013,1153,8,264,631,222,20,NaN,NaN,NaN,NaN,NaN
107014,464,NaN,15,48,115,126,116,30,9,NaN,4
107015,586,NaN,11,22,112,174,167,61,15,6,18


In [53]:
SQM_new.to_csv("SQM.csv")

#(9) RAUM

In [54]:
RAUM = xls.parse(
    8, index_col=[1], na_values=['-','.'],
    skiprows=[0,1,2,3,4,5,6],
    skip_footer=20
)

In [56]:
RAUM_new = makeNew(RAUM, cut=8, col_id=10)

In [57]:
RAUM_new.dtypes

Insgesamt           float64
1 Raum              float64
2 Räume             float64
3 Räume             float64
4 Räume             float64
5 Räume             float64
6 Räume             float64
7 und mehr Räume    float64
dtype: object

In [58]:
RAUM_new.head()

,Insgesamt,1 Raum,2 Räume,3 Räume,4 Räume,5 Räume,6 Räume,7 und mehr Räume
001002,10,NaN,3,3,3,NaN,NaN,NaN
001003,7,NaN,NaN,3,NaN,NaN,3,NaN
001004,32,5,NaN,9,11,3,5,NaN
001005,51,4,20,16,10,NaN,NaN,NaN
001006,41,NaN,3,16,16,3,3,NaN


In [59]:
RAUM_new.tail()

,Insgesamt,1 Raum,2 Räume,3 Räume,4 Räume,5 Räume,6 Räume,7 und mehr Räume
107011,314,3,4,16,46,116,84,46
107012,812,34,25,249,288,163,49,4
107013,1153,34,26,398,534,149,10,NaN
107014,464,NaN,3,17,96,181,115,53
107015,586,NaN,5,57,102,189,150,83


In [60]:
RAUM_new.to_csv("RAUM.csv")